In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [ ]:
@njit
def update_contour(candidate_loc, contour, domain):
    n = len(contour)
    x = candidate_loc[0]
    y = candidate_loc[1]
    domain[x, y] = 0
    contour[x, y] = 1
    neighbours = []
    if x > 0:
        neighbours.append([x-1, y])
    if y > 0:
        neighbours.append([x, y-1])
    if x < n-1:
        neighbours.append([x+1, y])
    if y < n-1:
        neighbours.append([x, y+1])
    for neigh in neighbours:
        if contour[neigh[0], neigh[1]] != 1 and neigh[0] in range(1, n-1) and neigh[1] in range(1, n-1):
            contour[neigh[0], neigh[1]] = 2
    candidates = [[x, y] for x in range(0,n) for y in range(0,n) if contour[x,y] == 2]
    return contour, candidates

def aggregate_candidate(candidates, domain, eta):
    a = [(domain[i, j])**eta for [i, j] in candidates]
    prob_cand = [c/np.sum(a) for c in a]
    agg_cand = candidates[np.random.choice(len(candidates), p = prob_cand)]
    return agg_cand

@njit
def jacobi_iteration(domain, contour):
    n = len(domain)
    next_domain = domain.copy()
    for x in range(1, n):
        for y in range(0, n):
            if contour[x, y] != 1:
                neighbours = []
                if contour[x-1, y] != 1:
                    neighbours.append(domain[x-1, y])
                if y > 0 and contour[x, y-1] != 1:
                    neighbours.append(domain[x, y-1])
                elif y == 0 and contour[x, -2] != 1:
                    neighbours.append(domain[x, -2])
                if x < n and contour[x+1, y] != 1:
                    neighbours.append(domain[x+1, y])
                if y < n and contour[x, y] != 1:
                    neighbours.append(domain[x, y+1])
                elif y == n and contour[x, 1] != 1:
                    neighbours.append(domain[x, 1])
                if not neighbours:
                    next_domain[x, y] = 0
                else:
                    next_domain[x, y] = sum(neighbours) / len(neighbours)
    return next_domain

@njit
def jacobi(domain, contour, eps = 1e-5):
    conv = 1
    while conv > eps:
        next_domain = jacobi_iteration(domain, contour)
        diff = np.abs(next_domain - domain)
        conv = np.max(diff)
        domain = next_domain
    return next_domain

In [ ]:
n = 30 # grid length (once squared)
eta = .1 # shape parameter 

domain = np.zeros((n, n))
domain[0, :] = 1
contour = np.zeros((n, n))

contour, candidates = update_contour([n-2, n//2], contour, domain)
domain = jacobi(domain, contour)
print(domain)

for _ in range(30):
    cand = aggregate_candidate(candidates, domain, eta)
    contour, candidates = update_contour(cand, contour, domain)
    domain = jacobi(domain, contour)

im = plt.imshow(domain, cmap='viridis_r')
plt.xlim(0, n-1)
plt.ylim(n-1, 0)
plt.show()

: 